In [ ]:
## Basic code by chatGPT

# Import necessary libraries
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np

# Load the dataset
data = datasets.load_landsat()

# Get the features (bands)
X = data.data

# Get the labels
y = data.target

# Calculate the means of each class
class_means = []
for i in np.unique(y):
    class_means.append(np.mean(X[y==i], axis=0))
    
# Calculate the overall mean
overall_mean = np.mean(X, axis=0)

# Calculate the between-class scatter matrix
Sb = np.zeros((X.shape[1], X.shape[1]))
for i in range(len(class_means)):
    Sb += np.outer((class_means[i] - overall_mean), (class_means[i] - overall_mean))

# Calculate the within-class scatter matrix
Sw = np.zeros((X.shape[1], X.shape[1]))
for i in np.unique(y):
    Sw += np.cov(X[y==i], rowvar=False)

# Calculate the eigenvalues and eigenvectors of (Sw^-1)*Sb
eigenvalues, eigenvectors = np.linalg.eig(np.dot(np.linalg.inv(Sw), Sb))

# Sort the eigenvectors by eigenvalues in descending order
eigenvectors = eigenvectors[:, np.argsort(eigenvalues)[::-1]]

# Project the data onto the top two eigenvectors (FLD)
X_fld = np.dot(X, eigenvectors[:, :2])

# Cluster the data using KMeans
kmeans = KMeans(n_clusters=5)
kmeans.fit(X_fld)

# Get the labels and plot the results
labels = kmeans.labels_
plt.scatter(X_fld[:, 0], X_fld[:, 1], c=labels, cmap='viridis')
plt.xlabel('FLD1')
plt.ylabel('FLD2')
plt.show()